# Lab 1: Value Iteration

*OpenAI gym FrozenLake environment*

Winter is here. You and your friends were tossing around a frisbee at the park
    when you made a wild throw that left the frisbee out in the middle of the lake.
    The water is mostly frozen, but there are a few holes where the ice has melted.
    If you step into one of those holes, you'll fall into the freezing water.
    At this time, there's an international frisbee shortage, so it's absolutely imperative that
    you navigate across the lake and retrieve the disc.
    However, the ice is slippery, so you won't always move in the direction you intend.
    The surface is described using a grid like the following

        SFFF
        FHFH
        FFFH
        HFFG

    S : starting point, safe
    F : frozen surface, safe
    H : hole, fall to your doom
    G : goal, where the frisbee is located

    The episode ends when you reach the goal or fall in a hole.
    You receive a reward of 1 if you reach the goal, and zero otherwise.
   

In this assignment, you will be implementing a value iteration algorithm

In [1]:
import gym
import numpy as np
print("numpy version", np.__version__)
print("gym version", gym.__version__)

numpy version 1.19.5
gym version 0.21.0


In [2]:
env=gym.make('FrozenLake-v1')

In [3]:
## DO NOT CHANGE THIS CELL

def argmax(env, V, pi, action,s, gamma):
    e = np.zeros(env.env.nA)
    for a in range(env.env.nA):                         # iterate for every action possible 
        q=0
        P = np.array(env.env.P[s][a])                   
        (x,y) = np.shape(P)                             # for Bellman Equation 
        
        for i in range(x):                              # iterate for every possible states
            s_= int(P[i][1])                            # S' - Sprime - possible succesor states
            p = P[i][0]                                 # Transition Probability P(s'|s,a) 
            r = P[i][2]                                 # Reward
            
            q += p*(r+gamma*V[s_])                      # calculate action_ value q(s|a)
            e[a] = q
            
    m = np.argmax(e) 
    action[s]=m                                         # Take index which has maximum value 
    pi[s][m] = 1                                        # update pi(a|s) 

    return pi

Your task is to complete the "bellman_opt_update" function below, which servces as a subroutine for the value iteration method.

In [4]:
def bellman_opt_update(env, V, s, gamma):  # update the stae_value V[s] by taking 

    #########################
    # Place your code here
    ##########################
    pi = np.zeros((env.env.nS, env.env.nA))
    e = np.zeros(env.env.nA)
    for a in range(env.env.nA):
        q = 0
        P = np.array(env.env.P[s][a])
        (x,y) = np.shape(P)

        for i in range(x):
            s_ = int(P[i][1])
            p = P[i][0]
            r = P[i][2]

            q += p*(r+gamma*V[s_])
        e[a] = q
    
    m = np.max(e)
    V[s] = m

In [5]:
## DO NOT CHANGE THIS CELL

def value_iteration(env, gamma, theta):
    V = np.zeros(env.env.nS)                              # initialize v(0) to arbitory value, my case "zeros"
    while True:
        delta = 0
        for s in range(env.env.nS):                       # iterate for all states
            v = V[s]
            bellman_opt_update(env, V, s, gamma)   # update state_value with bellman optimality update
            delta = max(delta, abs(v - V[s]))             # assign the change in value per iteration to delta  
        if delta < theta:                                       
            break                                         # if change gets to negligible 
                                                          # --> converged to optimal value         
    pi = np.zeros((env.env.nS, env.env.nA)) 
    action = np.zeros((env.env.nS))
    for s in range(env.env.nS):
        pi = argmax(env, V, pi, action, s, gamma)         # extract optimal policy using action value 
        
    return V, pi, action   # optimal value funtion, optimal policy with one-hot encoding (pi), optimal policy with discrete number (action)


In [7]:
## DO NOT CHANGE THIS CELL

gamma = 0.99
theta = 0.000001

env.seed(1)
V, pi, action = value_iteration(env, gamma, theta) # note "action" is optimal policy with discrete action number

#initialize episodic structure
num_episodes=1000;
episode_max_length=10000;


e=0
for i_episode in range(num_episodes):
    s = env.reset()
    for t in range(episode_max_length):
        s, reward, done, _ = env.step(action[s])
        if done:
            if reward == 1:
                e +=1
            break
print(" agent succeeded to reach goal {} out of {} Episodes using this policy ".format(e+1, num_episodes))
print(" success rate:", (e+1)/num_episodes)
env.close()

 agent succeeded to reach goal 746 out of 1000 Episodes using this policy 
 success rate: 0.746


### Save this notebook file (after you modify and run the code) and submit "lab1.ipynb" on eTL

Your success rate should be above 70%